In [1]:
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
from torchvision.io.image import read_image, write_png
import torchvision.transforms.functional as F
from pathlib import Path
from utils import setCWDToProjectDir
import torch
import os

nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
nnUNet_results is not defined and nnU-Net cannot be used for training or inference. If this is not intended behavior, please read documentation/setting_up_paths.md for information on how to set this up.


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
setCWDToProjectDir()

Using device: cuda


In [4]:
model_folder = "ECG-Digitiser/models/M3"
dataset_name = "Dataset500_Signals"

predictor = nnUNetPredictor(
        tile_step_size=0.5,
        use_gaussian=True,
        use_mirroring=True,
        perform_everything_on_device=True,
        device=device,
        verbose=True
    )

predictor.initialize_from_trained_model_folder(
        os.path.join(os.getcwd(),model_folder, "nnUNet_results", dataset_name, "nnUNetTrainer__nnUNetPlans__2d"),
        use_folds=['all'],
        checkpoint_name="checkpoint_final.pth",
    )
image_path = "ecg_example2.png"
image = read_image(image_path)
image = image[:3].to(torch.uint8)
tmp_dir = Path("tmp_prediction")
tmp_dir.mkdir(exist_ok=True)
tmp_image_path = tmp_dir / "input.png"
write_png(image, str(tmp_image_path))

print(f"Predicting on {tmp_image_path} and saving to {tmp_dir}")
prediction = predictor.predict_from_files(
        [[str(tmp_image_path)]],
        None,
        save_probabilities=False,
        overwrite=True,
        num_processes_preprocessing=1,  # Reduce to 1 process
        num_processes_segmentation_export=1  # Reduce to 1 proces
    )

print(f"Prediction shape: {prediction[0].shape}")

# Read result back

Predicting on tmp_prediction\input.png and saving to tmp_prediction
There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting image of shape torch.Size([1, 3, 704, 1241]):
perform_everything_on_device: True
Input shape: torch.Size([1, 3, 704, 1241])
step_size: 0.5
mirror_axes: (0, 1)
n_steps 3, image size is torch.Size([3, 1024, 1280]), tile_size [1024, 1280], tile_step_size 0.5
steps:
[[0], [0]]
move image to device cuda
preallocating results arrays on device cuda


100%|██████████| 3/3 [00:07<00:00,  2.47s/it]


Prediction done
sending off prediction to background worker for resampling

Done with image of shape torch.Size([1, 3, 704, 1241]):
Prediction shape: (3, 704, 1241)
